In [1]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re
import random

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

# Importar librerías para pausar la ejecución
from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

C:\Users\femes\AppData\Local\Temp\ipykernel_19860\3104618795.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
rotten = pd.read_csv("../INFO_PELIS/lista_pelis_2014_2020.csv", index_col = 0)

In [3]:
rotten.tail()

,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
72221,Action,Movie,Trapped,2020,3.0,tt30293906
72222,Action,Movie,Hridoy Jurey,2020,2.0,tt30613966
72223,Action,Movie,The call of the wild,2020,2.0,tt30701148
72224,Action,Movie,Special ops,2020,3.0,tt30877640
72225,Action,Movie,House of Stone,2020,12.0,tt31451622


In [4]:
lista_titulos= rotten["Titulo"].to_list()

In [5]:
lista_años= rotten["Año de estreno"].to_list()

In [6]:
list_busqueda_rotten = []
for titulo,año in zip(lista_titulos,lista_años):
    list_busqueda_rotten.append({titulo:año})

In [7]:
## Extraccion datos Rotten Tomatoes

url= "https://www.google.es/"


options = webdriver.FirefoxOptions()
options.page_load_strategy = "eager"
driver = webdriver.Firefox(options)

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()
sleep(3)

puntuacion_tomatoes = {}
contador = 68933
maximo = 69933

for peli in list_busqueda_rotten[contador:maximo]:
    for titulo,año in peli.items():
         
        busqueda = f"TOMATOMETER title: {titulo} - year: {año}"
        # print(busqueda)
        
        try:
                       
            buscador = driver.find_element("css selector",'#APjFqb')
            buscador.clear()
            buscador.send_keys(busqueda, Keys.ENTER)
            sleep(random.randint(4,6))

            try:
                puntuacion_tomatoes[titulo]=driver.find_element("css selector","#rso > div:first-child > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text
                                
            except:
                puntuacion_completa= driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text
                puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
                puntuacion_tomatoes[titulo]=puntuacion.group()
        
        except:
            puntuacion_tomatoes[titulo]="No existe puntuacion"



df_tomatoes = pd.Series(puntuacion_tomatoes,name="puntuacion")
df_tomatoes.index.name="titulo"
df_tomatoes.reset_index()
df_tomatoes.to_csv(f"{contador}_{maximo}_puntuacion_tomates_fedra.csv")